In [2]:
import pandas as pd
import json

In [3]:
def refactoring(data):
    new_data = {}
    for key, value in data.items():
        if isinstance(value, str) and '<' in value:
            value = int(value.split('<')[1])
        if value != '':
            new_data[key] = value
    return new_data


In [4]:
file = open('../data/raw.json', 'r', encoding='utf-8')
raw = json.load(file)
file.close()

data = []
for item in raw:
    jdata = json.loads(item['json'])
    station = jdata['arsopodatki']['postaja']
    for d in station:
        data.append(refactoring(d))

df = pd.DataFrame(data)

In [5]:
print(df.isnull().sum())

nadm_visina          0
datum_od             0
merilno_mesto        0
benzen           59707
ge_sirina         3073
sifra                0
datum_do             0
ge_dolzina        3073
pm2.5            10046
o3               35265
pm10             10046
co               61793
no2              37066
so2              56683
dtype: int64


In [6]:
kategoricni = df.select_dtypes(include=['object']).columns
print(kategoricni)
stevilski = df.select_dtypes(exclude=['object']).columns
print(stevilski)

Index(['datum_od', 'merilno_mesto', 'sifra', 'datum_do'], dtype='object')
Index(['nadm_visina', 'benzen', 'ge_sirina', 'ge_dolzina', 'pm2.5', 'o3',
       'pm10', 'co', 'no2', 'so2'],
      dtype='object')


In [7]:
for col in stevilski:
    df[col].fillna((df[col].mean()), inplace=True)

df_location = pd.get_dummies(df['merilno_mesto'])
df = pd.concat([df, df_location], axis=1).reindex(df.index)
df.pop('merilno_mesto')
df.pop('sifra') 
df.pop('datum_od')
df.pop('datum_do')

df

,nadm_visina,benzen,ge_sirina,ge_dolzina,pm2.5,o3,pm10,co,no2,so2,...,MB Vrbanski,MS Cankarjeva,MS Rakičan,NG Grčna,Novo mesto,Otlica,Ptuj,Rečica v I.Bistrici,Trbovlje,Zagorje
0,299,1.000000,46.065851,14.517454,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
1,305,1.093431,46.072399,14.491849,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
2,293,1.093431,46.037791,14.494001,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
3,388,1.093431,46.242115,14.366963,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
4,270,1.500000,46.559202,15.656191,35.610733,51.407339,42.277626,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64528,241,1.093431,46.131416,15.000906,32.000000,84.000000,34.000000,0.452701,8.000000,3.000000,...,0,0,0,0,0,0,0,0,0,1
64529,290,1.093431,46.144213,15.088097,28.000000,51.407339,29.000000,0.452701,24.180617,2.210446,...,0,0,0,0,0,0,0,0,0,0
64530,220,1.093431,45.802080,15.182060,33.000000,72.000000,35.000000,0.452701,6.000000,2.210446,...,0,0,0,0,1,0,0,0,0,0
64531,540,0.000000,45.561344,14.862816,12.000000,88.000000,14.000000,0.452701,24.180617,1.000000,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df[sorted(df)]
df

,CE Ljubljanska,CE bolnica,Hrastnik,Iskrba,Koper,Kranj,Krvavec,LJ Bežigrad,LJ Celovška,LJ Vič,...,benzen,co,ge_dolzina,ge_sirina,nadm_visina,no2,o3,pm10,pm2.5,so2
0,0,0,0,0,0,0,0,1,0,0,...,1.000000,0.452701,14.517454,46.065851,299,24.180617,51.407339,42.277626,35.610733,2.210446
1,0,0,0,0,0,0,0,0,1,0,...,1.093431,0.452701,14.491849,46.072399,305,24.180617,51.407339,42.277626,35.610733,2.210446
2,0,0,0,0,0,0,0,0,0,1,...,1.093431,0.452701,14.494001,46.037791,293,24.180617,51.407339,42.277626,35.610733,2.210446
3,0,0,0,0,0,1,0,0,0,0,...,1.093431,0.452701,14.366963,46.242115,388,24.180617,51.407339,42.277626,35.610733,2.210446
4,0,0,0,0,0,0,0,0,0,0,...,1.500000,0.452701,15.656191,46.559202,270,24.180617,51.407339,42.277626,35.610733,2.210446
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64528,0,0,0,0,0,0,0,0,0,0,...,1.093431,0.452701,15.000906,46.131416,241,8.000000,84.000000,34.000000,32.000000,3.000000
64529,0,0,1,0,0,0,0,0,0,0,...,1.093431,0.452701,15.088097,46.144213,290,24.180617,51.407339,29.000000,28.000000,2.210446
64530,0,0,0,0,0,0,0,0,0,0,...,1.093431,0.452701,15.182060,45.802080,220,6.000000,72.000000,35.000000,33.000000,2.210446
64531,0,0,0,1,0,0,0,0,0,0,...,0.000000,0.452701,14.862816,45.561344,540,24.180617,88.000000,14.000000,12.000000,1.000000


In [8]:
df.to_csv('../data/processed.csv', index=False)